![xarray Logo](http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png "xarray Logo")

# Introduction to Xarray

---

### Questions
1. What is Xarray?
2. How does Xarray fit in with Numpy and Pandas?
3. What is the CF convention and how do we use it with Xarray?

### Objectives
1. Create a `DataArray`.
2. Open netCDF data using XArray
3. Subset the data.
4. Write a CF-compliant netCDF file

## Overview

This notebook will introduce the basics of gridded, labeled data with Xarray. Since Xarray introduces additional abstractions on top of plain arrays of data, our goal is to show why these abstractions are useful and how they frequently lead to simpler, more robust code.

We'll cover these topics:

1. Create a `DataArray`.
2. Open netCDF data using XArray
3. Subset the data.
4. Simplified broadcasting using labeled dimension

## Prerequisites
This section was inspired by [this template](https://github.com/alan-turing-institute/the-turing-way/blob/master/book/templates/chapter-template/chapter-landing-page.md) of the wonderful [The Turing Way](https://the-turing-way.netlify.app/welcome.html) Jupyter Book.

Following your overview, tell your reader what concepts, packages, or other background information they'll **need** before learning your material. Tie this explicitly with links to other pages here in Foundations or to relevant external resources. Remove this body text, then populate the Markdown table, denoted in this cell with `|` vertical brackets, below, and fill out the information following. In this table, lay out prerequisite concepts by explicitly linking to other Foundations material or external resources, or describe generally helpful concepts.

Label the importance of each concept explicitly as **helpful/necessary**.

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Cartopy](../core/cartopy/01_Cartopy_Intro) | Necessary | |
| [Understanding of NetCDF](some-link-to-external-resource) | Helpful | Familiarity with metadata structure |
| Project management | Helpful | |

- **Experience level**: with relevant packages or general self-assessed experience as **beginner/user/expert**
- **Time to learn**: estimate in minutes or qualitatively as **long/medium/short**
- **System requirements**:
    - Populate with any system, version, or non-Python software requirements if necessary
    - Otherwise use the concepts table above and the Imports section below to describe required packages as necessary
    - If no extra requirements, remove the **System requirements** point altogether

---

## Imports
Begin your body of content with another `---` divider before continuing into this section, then remove this body text and populate the following code cell with all necessary Python imports **up-front**:

In [ ]:
import sys

## Xarray

Xarray expands on the capabilities on NumPy arrays, providing a lot of streamlined data manipulation. It is similar in that respect to Pandas, but whereas Pandas excels at working with tabular data, XArray is focused on N-dimensional arrays of data (i.e. grids). Its interface is based largely on the netCDF data model (variables, attributes, and dimensions), but it goes beyond the traditional netCDF interfaces to provide functionality similar to netCDF-java's Common Data Model (CDM). 

### `DataArray`

The `DataArray` is one of the basic building blocks of Xarray. It provides a NumPy ndarray-like object that expands to provide two critical pieces of functionality:

1. Coordinate names and values are stored with the data, making slicing and indexing much more powerful
2. It has a built-in container for attributes

In [ ]:
# Convention for import to get shortened namespace
import numpy as np
import xarray as xr

In [ ]:
# Create some sample "temperature" data
data = 283 + 5 * np.random.randn(5, 3, 4)
data

Here we create a basic `DataArray` by passing it just a numpy array of random data. Note that Xarray generates some basic dimension names for us.

In [ ]:
temp = xr.DataArray(data)
temp

We can also pass in our own dimension names:

In [ ]:
temp = xr.DataArray(data, dims=['time', 'lat', 'lon'])
temp

This is already improved upon from a numpy array, because we have names for each of the dimensions (or axes in NumPy parlance). Even better, we can take arrays representing the values for the coordinates for each of these dimensions and associate them with the data when we create the `DataArray`.

In [ ]:
# Use pandas to create an array of datetimes
import pandas as pd

times = pd.date_range('2018-01-01', periods=5)
times

In [ ]:
# Sample lon/lats
lons = np.linspace(-120, -60, 4)
lats = np.linspace(25, 55, 3)

When we create the `DataArray` instance, we pass in the arrays we just created:

In [ ]:
temp = xr.DataArray(data, coords=[times, lats, lons], dims=['time', 'lat', 'lon'])
temp

...and we can also set some attribute metadata:

In [ ]:
temp.attrs['units'] = 'kelvin'
temp.attrs['standard_name'] = 'air_temperature'

temp

Notice what happens if we perform a mathematical operaton with the `DataArray`: the coordinate values persist, but the attributes are lost. This is done because it is very challenging to know if the attribute metadata is still correct or appropriate after arbitrary arithmetic operations.

In [ ]:
# For example, convert Kelvin to Celsius
temp - 273.15

### Selection
We can use the `.sel` method to select portions of our data based on these coordinate values, rather than using indices (this is similar to the CDM).

In [ ]:
temp.sel(time='2018-01-02')

`.sel` has the flexibility to also perform nearest neighbor sampling, taking an optional tolerance:

In [ ]:
from datetime import timedelta

temp.sel(time='2018-01-07', method='nearest', tolerance=timedelta(days=2))

`.interp` ([docs](http://xarray.pydata.org/en/stable/interpolation.html)) works similarly to `.sel()`. Using `.interp()`, we can get an interpolated time series "forecast" for Boulder (40°N, 105°W) or your favorite latitude/longitude location. Note, xarray's interpolation functionality requires [SciPy](https://scipy.org/)!

In [ ]:
temp.interp(lon=-105, lat=40)

### Slicing with Selection

In [ ]:
temp.sel(
    time=slice('2018-01-01', '2018-01-03'), lon=slice(-110, -70), lat=slice(25, 45)
)

### `.loc`

All of these operations can also be done within square brackets on the `.loc` attribute of the `DataArray`. This permits a much more numpy-looking syntax, though you lose the ability to specify the names of the various dimensions. Instead, the slicing must be done in the correct order.

In [ ]:
# As done above
temp.loc['2018-01-02']

In [ ]:
temp.loc['2018-01-01':'2018-01-03', 25:45, -110:-70]

In [ ]:
# This *doesn't* work however:
# temp.loc[-110:-70, 25:45,'2018-01-01':'2018-01-03']

## Opening netCDF data
With its close ties to the netCDF data model, Xarray also supports netCDF as a first-class file format. This means it has easy support for opening netCDF datasets, so long as they conform to some of XArray's limitations (such as 1-dimensional coordinates).

In [ ]:
# Open sample North American Reanalysis data in netCDF format
ds = xr.open_dataset('NARR_19930313_0000.nc')
ds

This returns a `Dataset` object, which is a container that contains one or more `DataArray`s, which can also optionally share coordinates. We can then pull out individual fields:

In [ ]:
ds.isobaric1

or

In [ ]:
ds['isobaric1']

`Dataset`s also support much of the same subsetting operations as `DataArray`, but will perform the operation on all data:

In [ ]:
ds_1000 = ds.sel(isobaric1=1000.0)
ds_1000

In [ ]:
ds_1000.Temperature_isobaric

### Aggregation operations

Not only can you use the named dimensions for manual slicing and indexing of data, but you can also use it to control aggregation operations, like `sum`:

In [ ]:
u_winds = ds['u-component_of_wind_isobaric']
u_winds.std(dim=['x', 'y'])

Using the sample dataset, we can calculate the mean temperature profile (temperature as a function of pressure) over Colorado within this dataset. For this exercise, consider the bounds of Colorado to be:
 * x: -182km to 424km
 * y: -1450km to -990km
    
(37°N to 41°N and 102°W to 109°W projected to Lambert Conformal projection coordinates)

In [ ]:
temps = ds.Temperature_isobaric
co_temps = temps.sel(x=slice(-182, 424), y=slice(-1450, -990))
prof = co_temps.mean(dim=['x', 'y'])
prof

### Resources

There is much more in the Xarray library. To learn more, visit the [Xarray Documentation](http://xarray.pydata.org/en/stable/index.html)